In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler 

In [ ]:
data = pd.read_csv('data.csv')
data = data.dropna()
data


import pandas as pd

# Charger le fichier Excel
df = pd.read_excel('Taux_Remplissage_ComCom_Nebbiu.xlsx')

# Filtrer les données pour le village sélectionné
village_selectionne = "sorio"
df_village = df[df['Village'] == village_selectionne]

# Créer le tableau
tableau = df_village[['Village', 'Poubelle', 'GPS', 'Remplissage', 'CoeffTouriste']].values.tolist()

# Afficher le tableau
for ligne in tableau:
    print(ligne)

In [ ]:
scaler = StandardScaler()
data[['niveau_remplissage', 'densite_population', 'affluence_touristique']] = scaler.fit_transform(data[['niveau_remplissage', 'densite_population', 'affluence_touristique']])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

In [ ]:
new_data = pd.DataFrame({'niveau_remplissage': [0.8], 'densite_population': [1.2], 'affluence_touristique': [0.5]})
scaled_data = scaler.transform(new_data)
prediction = model.predict(scaled_data)
print(f'Prédiction de la fréquence de collecte : {prediction}')